---
# Automatically generate new Python files from the Spoonacular API info page
---

In [42]:
import time
import json
import re
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

---
# Scrape the API documentation

## Start the Selenium browser (Chrome)

In [2]:
# Start a new instance of Chrome
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--no-sandbox')
browser = webdriver.Chrome('/usr/local/bin/chromedriver', chrome_options=chrome_options)
url = "https://spoonacular.com/food-api"
browser.get(url)
time.sleep(1)

# Scroll down to list of endpoints (necessary?)
elem = browser.find_element_by_tag_name("body")
no_of_pagedowns = 8
while no_of_pagedowns:
    elem.send_keys(Keys.PAGE_DOWN)
    time.sleep(2)
    no_of_pagedowns-=1
    
print("Waiting for endpoints to load...")
time.sleep(5)

Waiting for endpoints to load...


## Scrape documentation for each endpoint

In [3]:
def getEndpointInfo(save=True, filename='spoonacular_api_info'):
    endpoints = []
    m,p = 'mashape-', 'parameter-'
    regex = re.compile(r'required|optional', re.VERBOSE)    
    doc = browser.find_element_by_class_name(m+'doc')
    groups = doc.find_elements_by_class_name(m+'group')
    num_eps = len(doc.find_elements_by_class_name(m+'endpoint'))

    n = 0
    for group in groups:
        print(10*'-' + group.find_element_by_class_name(m+'group-header').text + 10*'-')
        for ep in group.find_elements_by_class_name(m+'endpoint'):            
            if n!= 36: # Kludge for bug on Spoonacular site
                ep.click()  # Must click on endpoint to expand its documentation
            time.sleep(0.5)
            print('**'+ep.find_element_by_class_name(m+'endpoint-name').text+'**')
                                                
            # Get info on the endpoint name and route
            endpoint = {}
            endpoint['name'] = ep.find_element_by_class_name(m+'endpoint-name').text
            print('({n}/{t}) "{name}"'.format(name=endpoint['name'], n=n+1, t=num_eps))
            content = ep.find_element_by_class_name(m+'endpoint-content')
            route = content.find_element_by_class_name(m+'endpoint-route').text.split(' /')
            endpoint['method'] = route[0]
            endpoint['uri'] = route[1]
            endpoint['category'] = group.find_element_by_class_name(m+'group-header').text
            endpoint['description'] = ep.find_element_by_tag_name('p').text
            time.sleep(1.5)            
            
            # Get parameter info for the endpoint
            parameters = {'route_params': [], 'params': []}
            assign_to_route_params = True        
            for div in content.find_elements_by_tag_name('div'):
                if div.get_attribute('class') == 'mashape-parameter-header':
                    print(div.text)
                    if div.text == 'Parameters':
                        assign_to_route_params = False

                if div.get_attribute('class') == 'mashape-parameter':                
                    parameter = {}    
                    if div.text != '':
                        for s in div.find_elements_by_tag_name('span'):
                            var = s.get_attribute('class').split(m+p,1)[-1].lower()
                            if 'condition' in var:
                                var = 'condition'
                                val = regex.search(s.text).group()
                            elif 'example' in var:
                                val = s.text.split('Example:',1)[-1]
                            elif 'name' in var:
                                if s.text in ['id']:
                                    val = val + '_'
                            else:
                                val = s.text
                            parameter[var] = val
                        print("\t{name}: {type}".format(name=parameter['name'], type=parameter['type']))
                    else:
                        print("\t\t***MISSING PARAMETER***")
                        parameter = {'MISSING': True}                

                    if assign_to_route_params:
                        parameters['route_params'].append(parameter)
                    else:
                        parameters['params'].append(parameter)
                        
                endpoint['parameters'] = parameters

                # Get info on example endpoint response
                try:
                    endpoint['response'] = content.find_element_by_class_name(m+'example').text
                except:
                    endpoint['response'] = None                                    

            # Add info on current endpoint to list of all endpoints            
            endpoints.append(endpoint)
            n += 1
            
    if save:
        with open(filename + '.json', 'w') as outfile:
            json.dump(endpoints, outfile)                
                
    return endpoints                               
                                    

In [4]:
info = getEndpointInfo()
info[-1]

----------Extract----------
**Analyze a Recipe Search Query**
------------------------------
(1/53) "Analyze a Recipe Search Query"
Route Parameters
	q: STRING
**Analyze Recipe Instructions**
------------------------------
(2/53) "Analyze Recipe Instructions"
Parameters
	instructions: STRING
**Detect Food in Text**
------------------------------
(3/53) "Detect Food in Text"
Parameters
	text: STRING
**Extract Recipe from Website**
------------------------------
(4/53) "Extract Recipe from Website"
Route Parameters
	forceExtraction: BOOLEAN
	url: STRING
**Parse Ingredients**
------------------------------
(5/53) "Parse Ingredients"
Route Parameters
	includeNutrition: BOOLEAN
Parameters
	ingredientList: STRING
	servings: NUMBER
----------Search----------
**Autocomplete Ingredient Search**
------------------------------
(6/53) "Autocomplete Ingredient Search"
Route Parameters
	intolerances: STRING
	metaInformation: BOOLEAN
	number: NUMBER
	query: STRING
**Autocomplete Recipe Search**
-----

		***MISSING PARAMETER***
		***MISSING PARAMETER***
		***MISSING PARAMETER***
		***MISSING PARAMETER***
		***MISSING PARAMETER***
		***MISSING PARAMETER***
		***MISSING PARAMETER***
		***MISSING PARAMETER***
**Search Site Content**
------------------------------
(25/53) "Search Site Content"
Route Parameters
	query: STRING
----------Compute----------
**Classify a Grocery Product**
------------------------------
(26/53) "Classify a Grocery Product"
**Classify Cuisine**
------------------------------
(27/53) "Classify Cuisine"
Parameters
	ingredientList: STRING
	title: STRING
**Classify Grocery Products (Batch)**
------------------------------
(28/53) "Classify Grocery Products (Batch)"
**Convert Amounts**
------------------------------
(29/53) "Convert Amounts"
Route Parameters
	ingredientName: STRING
	sourceAmount: NUMBER
	sourceUnit: STRING
	targetUnit: STRING
**Create Recipe Card**
------------------------------
(30/53) "Create Recipe Card"
Parameters
	author: STRING
	backgroundColor

{'name': 'Talk to a chatbot',
 'method': 'GET',
 'uri': 'food/converse',
 'category': 'Chat',
 'description': 'This endpoint can be used to have a conversation about food with the spoonacular chat bot. Use the chat suggests endpoint to show your user what he or she can say.',
 'parameters': {'route_params': [{'name': 'contextId',
    'type': 'STRING',
    'condition': 'optional',
    'description': 'An arbitrary globally unique id for your conversation. The conversation can contain states so you should pass your context id if you want the bot to be able to remember the conversation.',
    'example': '342938'},
   {'name': 'text',
    'type': 'STRING',
    'condition': 'required',
    'description': 'The request/question/answer from the user to the chat bot.',
    'example': 'donut recipes'}],
  'params': []},
 'response': '{"answerText":"Here are some donut recipes for you.","media":[{"title":"Homemade Chinese Doughnuts","image":"https://spoonacular.com/recipeImages/797495-312x231.jpg"

## Scrape the Quota information for each endpoint

In [4]:
import requests
from bs4 import BeautifulSoup

In [5]:
# Load the HTML from Spoonacular
url = 'https://spoonacular.com/food-api/docs/quotas'
page = requests.get(url)
html = BeautifulSoup(page.text, "html.parser")

In [55]:
# Extract the table with quota information
table = html.find('table')
rows = table.findAll('tr')
all_text = []
for tr in rows:
    cols = tr.findAll('td')
    text_data = []
    for td in cols:
        text = '0' if td.text == '-' else td.text        
        text_data.append(text)
    if text_data:
        all_text.append(text_data)

In [97]:
# Format the quota information as a JSON object
quotas = []
for row in all_text:
    quota = {}
    quota['endpoint'] = row[0]
    for category, value in zip(['requests', 'tinyrequests', 'results'], row[1:]):
        amount, qualifier = zip((value + ' ').split(' ', 1))
        quota[category] = {'amount': amount[0].strip(), 'qualifier': qualifier[0].strip()}
    quotas.append(quota)

In [98]:
quotas[2]

{'endpoint': 'Classify Grocery Products (Batch)',
 'requests': {'amount': '0', 'qualifier': ''},
 'tinyrequests': {'amount': '1', 'qualifier': 'per product'},
 'results': {'amount': '0', 'qualifier': ''}}

---
# Generate the Python wrapper from scraped API documentation

## Load the Spoonacular API data

In [146]:
all_info = json.load(open('spoonacular_api_info.json', 'rb'))

In [159]:
def formatMethodName(name):
    name = name.lower().replace('(','').replace(')','')
    return name.replace(' ','_')

def formatDocString(docStr, width=72, indent=0):
    words = docStr.split(' ')
    txt = words[0] + ' '
    for word in words[1:]:
        if len(txt.splitlines()[-1]) > (width-4*indent):
            txt = txt.strip()
            txt += '\n\t\t'
        txt += word + ' '
    return txt.strip()
        
def formatVarName(name):
    """ Avoid Python-protected names """
    if name in ['id']:
        return name + '_'
    return name

def formatArgList(parameters):
    allParams = []
    for p in parameters['params']:
        allParams.append(p)
    for p in parameters['route_params']:        
        allParams.append(p)
    if len(allParams) == 0:
        return "self"
    assert len(allParams) < 10, "Var list can't be greater than 10."
    
    # Deal with optional variables
    required, optional = [], []
    for var in allParams:
        if var['condition'] == 'required':
            required.append(var['name'])
        else:
            optional.append("{name}=None".format(name=var['name']))            
            
    return ("self, " + ", ".join(required + optional)).strip(', ')

def formatQuery(uri, params):
    txt = "{"
    for q in params:
        if not paramInUri(uri, q['name']):
            txt += '"{}": {}, '.format(q['name'], q['name'])
    return txt.strip(', ') + "}"       

def extractEndpointInfo(epInfo):
    return epInfo['name'], epInfo['description'], epInfo['parameters'], epInfo['method'], epInfo['uri']

def indentMethod(txt, indent):
    # Indent the method (optional) and replace tabs with spaces
    final = ""
    for line in txt.split('\n'):
        final += "\t"*indent + line + '\n'
    return final.replace('\t', 4*' ')

def paramInUri(uri, paramName):    
    return "{{{p}}}".format(p=paramName) in uri          

def formatUri(uri, params):
    vals = ", ".join(["{k}={k}".format(k=p['name']) for p in params if paramInUri(uri, p['name'])])
    if vals:   
        return '.format({vals})'.format(vals=vals)                   
    return ''
        

def writeEndpointMethod(epInfo, indent=0):
    name, docStr, params, method, uri = extractEndpointInfo(epInfo)
    methodName = formatMethodName(name)
    
    txt = 'def {name}({args}):\n'.format(name=methodName, args=formatArgList(params))
    txt += '\t""" {doc}\n'.format(doc=formatDocString(docStr, 54, indent))
    txt += '\t\thttps://market.mashape.com/spoonacular/recipe-food-nutrition#{name}\n'.format(name=name.lower().replace(' ','-'))
    txt += '\t"""\n'
    txt += '\tendpoint = "{uri}"{f}\n'.format(uri=uri, f=formatUri(uri, params['route_params']))
    txt += '\turl_query = {q}\n'.format(q=formatQuery(uri, params['params']))
    txt += '\turl_params = {q}\n'.format(q=formatQuery(uri, params['route_params']))
    txt += '\treturn self._make_request(endpoint, method="{m}", query_=url_query, params_=url_params)'.format(m=method)
    
    # Indent the method (optional) and replace tabs with spaces    
    return indentMethod(txt, indent)

def formatTestArgs(epInfo):
    testValues = {}
    for param_type in list(epInfo['parameters'].values())[::-1]:
        for param in param_type:
            testValues[param['name']] = param['example']
            
    txt = '{'
    for key,val in testValues.items():    
        txt += "'{k}': '{v}', ".format(k=key, v=val)

    return txt.strip(', ') + '}'

def writeEndpointUnitTest(epInfo, indent=0):
    name, docStr, params, method, uri = extractEndpointInfo(epInfo)
    methodName = formatMethodName(name)
    testArgs = formatTestArgs(epInfo)
    
    txt = 'def test_{name}(self):\n'.format(name=methodName)
    txt += '\t"""Test the \'{name}\' endpoint ({method})"""\n'.format(name=name.lower(), method=method.upper())
    txt += '\tmsg = "Response status is not 200"\n'
    txt += '\ttestArgs = {args}\n'.format(args=formatTestArgs(epInfo))
    txt += '\tresponse = self.api.{name}(**testArgs)\n'.format(name=methodName)
    txt += '\tself.assertEqual(response.status_code, 200, msg)'
        
    return indentMethod(txt, indent)
    

In [162]:
print(writeEndpointMethod(all_info[7]))

def get_comparable_products(self, upc):
    """ Find comparable products to the given one.
        https://market.mashape.com/spoonacular/recipe-food-nutrition#get-comparable-products
    """
    endpoint = "food/products/upc/{upc}/comparable".format(upc=upc)
    url_query = {}
    url_params = {}
    return self._make_request(endpoint, method="GET", query_=url_query, params_=url_params)



## Write the endpoint methods and unit tests

In [163]:
method_counter = 0
txt = {}
# -------- METHODS -------- #
txt['m'] = ''
for category_name in list(set(ep['category'] for ep in all_info)):
    print('-'*30)
    print(category_name)

    # Write multiple endpoint methods to a Python file
    txt['m'] += '\n\t""" {f1} {cat} Endpoints {f1} """\n'.format(cat=category_name, f1='-'*15)
    skipped_methods = []
    for n,ep in enumerate(all_info):
        if ep['category'] == category_name:
            try:
                assert (ep['name'] not in ['Visualize Menu Item Nutrition', 'Visualize Product Nutrition','Get Menu Item Information']), 'Skipping'
                m = writeEndpointMethod(ep, indent=1)
                txt['m'] += "\n" + m
                method_counter += 1
            except:
                skipped_methods.append(ep['name'])

# -------- UNIT TESTS -------- #
txt['t'] = ''
for category_name in list(set(ep['category'] for ep in all_info)):
    print('-'*30)
    print(category_name)
        
    txt['t'] += '\n\t""" {f1} {cat} Endpoints {f1} """\n'.format(cat=category_name, f1='-'*15)

    # Write multiple unit tests to a Python file
    skipped_tests = []
    for n,ep in enumerate(all_info):
        if ep['category'] == category_name:
            try:
                m = writeEndpointUnitTest(ep, indent=1)
                txt['t'] += "\n" + m
            except:
                skipped_tests.append(ep['name'])

# Check that the same endpoints were skipped for methods and tests
assert skipped_methods == skipped_tests, 'Uh oh, skipped different endpoints for methods and tests'

# Save methods to a Python file
filename = 'spoonacular_api_methods.py'
with open(filename, 'w+') as pyfile:
    pyfile.write(txt['m'])    

# Save tests to a Python file
filename =  'spoonacular_api_tests.py'
with open(filename, 'w+') as pyfile:
    pyfile.write(txt['t'])    

print('\nDone. Generated methods and tests for {n} endpoints.'.format(n=method_counter))

------------------------------
Compute
------------------------------
Search
------------------------------
Chat
------------------------------
Data
------------------------------
Extract
------------------------------
Compute
------------------------------
Search
------------------------------
Chat
------------------------------
Data
------------------------------
Extract

Done. Generated methods and tests for 43 endpoints.


In [26]:
print(m)


    def autocomplete_ingredient_search(self, query, intolerances=None, metaInformation=None, number=None):
        """ Autocomplete a search for an ingredient.
            https://market.mashape.com/spoonacular/recipe-food-nutrition#autocomplete-ingredient-search
        """
        endpoint = "food/ingredients/autocomplete"
        query = {}
        params = {"intolerances": intolerances, "metaInformation": metaInformation, "number": number, "query": query}
        return self._make_request(endpoint, method="GET", query_=query, params_=params)

    def autocomplete_recipe_search(self, query, number=None):
        """ Autocomplete a partial input to possible recipe names.
            https://market.mashape.com/spoonacular/recipe-food-nutrition#autocomplete-recipe-search
        """
        endpoint = "recipes/autocomplete"
        query = {}
        params = {"number": number, "query": query}
        return self._make_request(endpoint, method="GET", query_=query, params_=params)

   